# Making queries

데이터 모델을 생성하면 개체를 생성, 삭제, 수정, 조회 할 수 있는 데이터베이스 추상화 API를 제공한다. 본 문서는 데이터베이스 추상화 API를 사용하는 방법을 설명한다. 다양한 모델 lookup 연산에 대한 자세한 설명은 data model reference를 참고해라

이 가이드 전체에서 우리는 웹로그 어플리케이션을 구성하는 다음 모델을 참조할 것이다.

```python
from django.db import models

class Blog(models.Model):
    name = models.CharField(max_length=100)
    tagline = models.TextField()
    
    def __str__(self):
        return self.name
    
class Author(models.Model):
    name = models.CharField(max_length=200)
    email = models.EmailField()
    
    def __str__(self):
        return self.name
    
class Entry(models.Model):
    blog = models.ForeignKey(Blog, on_delete=models.CASCADE)
    headline = models.CharField(max_length=255)
    body_text = models.TextField()
    pub_date = models.DateField()
    mod_date = models.DateField()
    authros = models.ManyToManyField(Author)
    n_comments = models.IntegerField()
    n_pingbacks = models.IntegerField()
    rating = models.IntegerField()
    
    def __str__(self):
        return self.headline
```

In [7]:
MYPROJECT = '/d/01.Workspace/django-practice'
import os, sys
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

** 커멘드 쉘 또는 파이참 등의 실행 환경에서 데이터베이스 마이그레이션을 수행 한 후 아래 코드를 테스트 한다**

## Creating objects

데이터베이스 테이블 데이터를 파이썬 개체로 표현하기 위해 장고는 직관적인 체계를 사용한다. 모델 클래스는 테이블을 나타낸다. 그리고 모델 클래스의 개체는 테이블에 있는 특정한 개체를 나타낸다

개체를 생서하기 위해서 모델 클래스에 키워드 매개 변수를 사용해서 인스턴스화한 후 데이터베이스에 저장하기 위해서 save()를 호출해라

모델은 mysite/blog/models.py에 있다고 가정한다. 예제는 다음과 같다

In [4]:
from making_queries.models import Blog

b = Blog(name='Beatles Blog', tagline='All the latest Beatles news.')
b.save()


위 예제는 INSERT SQL 문을 수행한다. 장고는 명시적으로 save()를 호출하기 전까지 실제 데이터베이스에 데이터를 저장하지 않는다.

save() 메소드는 반환 값이 없다.

> 여기서는 설명하지 않지만 save()는 다양한 부가 옵션이 있다. 자세한 설명은 [save()](https://docs.djangoproject.com/en/2.0/ref/models/instances/#django.db.models.Model.save) 문서를 참고해라.
>
> 한 번에 개체를 생성하려면 create() 메소드를 사용해라



## Saving changes to objects

이미 데이터베이스에 있는 개체의 변화를 저장하기 위해서 save()를 사용해라.

이미 데이터베이스에 저장된 b5 인스턴스가 주어졌단고 하면 이 예제는 b5의 이름을 변경하고 데이터베이스의 레코드를 갱신한다.

In [ ]:
b5.name = 'New name'
b5.save()

예제 코드는 UPDATE SQL 문을 수행한다. 장고는 명시적으로 save()를 호출하기 전까지 데이터베이스를 실행하지 않는다.

### Saving ForeignKey and ManyToManyField fields

ForeignKey 필드를 갱신 작업은 

In [5]:
from django.db import connection
print (connection.queries)

[{'sql': 'BEGIN', 'time': '0.000'}, {'sql': 'INSERT INTO "making_queries_blog" ("name", "tagline") VALUES (\'Beatles Blog\', \'All the latest Beatles news.\')', 'time': '0.000'}, {'sql': 'BEGIN', 'time': '0.000'}, {'sql': 'INSERT INTO "making_queries_blog" ("name", "tagline") VALUES (\'Beatles Blog\', \'All the latest Beatles news.\')', 'time': '0.001'}]


In [6]:
print(Blog.objects.filter(name='Beatles Blog').query)

SELECT "making_queries_blog"."id", "making_queries_blog"."name", "making_queries_blog"."tagline" FROM "making_queries_blog" WHERE "making_queries_blog"."name" = Beatles Blog
